# Fetching observations from MET data base services

This notebook serves as sandbox for fetching data from frost.met, havvarsel-frost.met and thredds.met

See https://api.met.no/ for all interfaces and possible sources!!


## Havvarsel frost
Havvarsel frost delivers so far `temperature` measurements originated from badevann.no at seven beaches in the south of Norway (glider data integrated in the meantime, but not yet used here!). 
> Documentation:
> API documentation for obs/badevann https://havvarsel-frost.met.no/docs/apiref#/obs%2Fbadevann/obsBadevannGet 
> Datastructure described on https://havvarsel-frost.met.no/docs/dataset_badevann


## Thredds
Holds netcdf files with the a bunch of different data

> See the catalog: https://thredds.met.no/thredds/catalog.html

We primarily use the `Ocean and Ice/met.no (OLD) ROMS NorKyst800m coastal forecasting system` to get the forecasted water temperatures


In [7]:
# Importing general libraries
import sys
import json
import datetime, time 
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import netCDF4
import pyproj as proj

from importlib import reload

In [8]:
start_time = datetime.datetime.strptime("2021-10-10T00:00", "%Y-%m-%dT%H:%M")
end_time = datetime.datetime.strptime("2021-10-12T12:00", "%Y-%m-%dT%H:%M")

In [9]:
times = pd.date_range(start_time, end_time, freq="H")
times = times.tz_localize("UTC")
timeseries = pd.DataFrame(times, columns=["time"])

In [10]:
import HavvarselFrostImporter
havvarsel = HavvarselFrostImporter.HavvarselFrostImporter(start_time,end_time)
havvarsel_info, havvarsel_timeseries = havvarsel.data(100)

- 12:51:04 -
Trying https://havvarsel-frost.met.no/api/v1/obs/badevann/get?time=2021-10-10T00:00:00Z/2021-10-12T12:00:00Z&incobs=true&buoyids=100&parameter=temperature
  buoyid    parameter     name        lon        lat
0    100  temperature  Sørenga  10.749700  59.900700


In [11]:
lon = float(havvarsel_info["lon"])
lat = float(havvarsel_info["lat"])
start_time = havvarsel_timeseries.index[0].to_pydatetime().replace(tzinfo=None)
end_time = havvarsel_timeseries.index[-1].to_pydatetime().replace(tzinfo=None)

In [12]:
import NorKystImporter
norkyst = NorKystImporter.NorKystImporter(start_time, end_time)

In [13]:
norkyst_df = norkyst.norkyst_data("temperature",lon,lat)

- 12:51:05 -
Coordinates model (x,y= 544,180): 59.898461480861215, 10.740797814175055
- 12:51:08 -
Processing  https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2021101000.nc
- 12:51:09 -
Processing  https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2021101100.nc
- 12:51:10 -
Processing  https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2021101200.nc


In [14]:
import PPImporter
pp = PPImporter.PPImporter(start_time, end_time)

In [15]:
pp_df = pp.pp_data(["air_temperature_2m", "wind_direction_10m"],lon,lat)

Filename timestamp based on start_time: 2021101000
Filename timestamp based on end_time: 2021101211
- 12:51:11 -
Processing https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T00Z.nc
Coordinates model (x,y= 660,767): 59.89720311267746, 10.745418590272147
- 12:51:14 -
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T00Z.nc
- 12:51:15 -
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T01Z.nc
- 12:51:15 -
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T02Z.nc
- 12:51:15 -
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T03Z.nc
- 12:51:15 -
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T04Z.nc
- 12:51:16 -
Processing  https://thredds.met.no/thredds/dodsC

Additional information 

In [30]:
import netCDF4
nc = netCDF4.Dataset("https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2022010100.nc")
import pyproj as proj
for var in ['polar_stereographic','projection_stere','grid_mapping']:
    if var in nc.variables.keys():
        try:
            proj1 = nc.variables[var].proj4
        except:
            proj1 = nc.variables[var].proj4string
p1 = proj.Proj(str(proj1))
for var in ['latitude','lat']:
    if var in nc.variables.keys():
        lat1 = nc.variables[var][:]
for var in ['longitude','lon']:
    if var in nc.variables.keys():
        lon1 = nc.variables[var][:]
xproj1,yproj1 = p1(lon1,lat1)


In [38]:
df = pd.read_csv("data/badevann-buoys.csv")

df["NorKyst-x"] = len(df)*[0]
df["NorKyst-y"] = len(df)*[0]

In [47]:
for b in range(len(df)):
    lon = df.iloc[b]["lon"]
    lat = df.iloc[b]["lat"]

    xp1,yp1 = p1(lon,lat)

    # find coordinate of gridpoint to analyze (only wet cells)
    h = np.array(nc["h"])
    land_value = h.min()
    land_mask = np.where((h!=land_value),0,1)
    distances = (xproj1-xp1)**2 + (yproj1-yp1)**2 + land_mask*1e12
    y1, x1 = np.unravel_index(distances.argmin(), distances.shape)
    
    df.at[b, "NorKyst-x"] = x1
    df.at[b, "NorKyst-y"] = y1
    

In [49]:
df.to_csv("data/badevann-buoys_extended.csv")